### Singular Value Decomposition

So far in this lesson, you have gained some exposure to Singular Value Decomposition.  In this notebook, you will get some hands on practice with this technique.

Let's get started by reading in our libraries and setting up the data we will use throughout this notebook.

`1.` Run the cell below to create the **user_movie_subset** dataframe.  This will be the dataframe you will use for the first part of this notebook.

**Note: Unstacking the matrix here could take ~10 mins to run.**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
print(user_movie_subset)

movie_id  73486  75314  68646  99685
user_id                             
265        10.0   10.0   10.0   10.0
1023       10.0    4.0    9.0   10.0
1683        8.0    9.0   10.0    5.0
6571        9.0    8.0   10.0   10.0
11639      10.0    5.0    9.0    9.0
13006       6.0    4.0   10.0    6.0
14076       9.0    8.0   10.0    9.0
14725      10.0    5.0    9.0    8.0
23548       7.0    8.0   10.0    8.0
24760       9.0    5.0    9.0    7.0
28713       9.0    8.0   10.0    8.0
30685       9.0   10.0   10.0    9.0
34110      10.0    9.0   10.0    8.0
34430       5.0    8.0    5.0    8.0
35150      10.0    8.0   10.0   10.0
43294       9.0    9.0   10.0   10.0
46849       9.0    8.0    8.0    8.0
50556      10.0    8.0    1.0   10.0
51382       5.0    6.0   10.0   10.0
51410       8.0    7.0   10.0    7.0


`2.` Now that you have the **user_movie_subset** matrix, use this matrix to correctly match each key to the correct value in the dictionary below.  Use the cells below the dictionary as necessary.

In [10]:
# match each letter to the best statement in the dictionary below - each will be used at most once
a = 20
b = 68646
c = 'The Godfather'
d = 'Goodfellas'
e = 265
f = 30685
g = 4

sol_1_dict = {
    'the number of users in the user_movie_subset': a,# letter here,
    'the number of movies in the user_movie_subset': g,# letter here,
    'the user_id with the highest average ratings given': e,# letter here,
    'the movie_id with the highest average ratings received': b,# letter here,
    'the name of the movie that received the highest average rating': c# letter here
}


#test dictionary here
t.test1(sol_1_dict)

That's right!  There are 20 users in the dataset, which is given by the number of rows. There are 4 movies in the dataset given by the number of columns.  You can find the movies or users with the highest average ratings by taking the mean of each row or column.  Using the movies table, you can find the movie names associated with each id.  This shows the top rated movie is The Godfather!


In [2]:
user_movie_subset.shape

(20, 4)

In [9]:
# Cell for work
# user with the highest average rating
user_movie_subset.mean(axis=1)

# movie with highest average rating
user_movie_subset.mean(axis=0)

# list of movie names
movies[movies.movie_id==68646]['movie']
# users by movies


3706    The Godfather (1972)
Name: movie, dtype: object

Now that you have a little more context about the matrix we will be performing Singular Value Decomposition on, we're going to do just that.  To get started, let's remind ourselves about the dimensions of each of the matrices we are going to get back.   Essentially, we are going to split the **user_movie_subset** matrix into three matrices:

$$ U \Sigma V^T $$


`3.` Given what you learned in the previous parts of this lesson, provide the dimensions for each of the matrices specified above using the dictionary below.

In [11]:
# match each letter in the dictionary below - a letter may appear more than once.
a = 'a number that you can choose as the number of latent features to keep'
b = 'the number of users'
c = 'the number of movies'
d = 'the sum of the number of users and movies'
e = 'the product of the number of users and movies'

sol_2_dict = {
    'the number of rows in the U matrix': b,# letter here, 
    'the number of columns in the U matrix': a,# letter here, 
    'the number of rows in the V transpose matrix': a,# letter here, 
    'the number of columns in the V transpose matrix': c# letter here
}

#test dictionary here
t.test2(sol_2_dict)

That's right!  We will now put this to use, so you can see how the dot product of these matrices come together to create our user item matrix.  The number of latent features will control the sigma matrix as well, and this will a square matrix that will at most be the minimum of the number of users and number of movies (in our case the minimum is the 4 movies).


Now let's verify the above dimensions by performing SVD on our user-movie matrix.

`4.` Below you can find the code used to perform SVD in numpy.  You can see more about this functionality in the [documentation here](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.linalg.svd.html).  What do you notice about the shapes of your matrices?  If you try to take the dot product of the three objects you get back, can you directly do this to get back the user-movie matrix?

In [12]:
u, s, vt = np.linalg.svd(user_movie_subset) # perform svd here on user_movie_subset
s.shape, u.shape, vt.shape

((4,), (20, 20), (4, 4))

In [13]:
# Run this cell for our thoughts on the questions posted above
t.question4thoughts()

Looking at the dimensions of the three returned objects, we can see the following:

 1. The u matrix is a square matrix with the number of rows and columns equaling the number of users. 

 2. The v transpose matrix is also a square matrix with the number of rows and columns equaling the number of items.

 3. The sigma matrix is actually returned as just an array with 4 values, but should be a diagonal matrix.  Numpy has a diag method to help with this.  

 In order to set up the matrices in a way that they can be multiplied together, we have a few steps to perform: 

 1. Turn sigma into a square matrix with the number of latent features we would like to keep. 

 2. Change the columns of u and the rows of v transpose to match this number of dimensions. 

 If we would like to exactly re-create the user-movie matrix, we could choose to keep all of the latent features.


`5.` Use the thoughts from the above question to create u, s, and vt with four latent features.  When you have all three matrices created correctly, run the test below to show that the dot product of the three matrices creates the original user-movie matrix.  The matrices should have the following dimensions:

$$ U_{n x k} $$

$$\Sigma_{k x k} $$

$$V^T_{k x m} $$

where:

1. n is the number of users
2. k is the number of latent features to keep (4 for this case)
3. m is the number of movies


In [15]:
# Change the dimensions of u, s, and vt as necessary to use four latent features
# update the shape of u and store in u_new
u_new = u[:, :len(s)]# change the shape of u here

# update the shape of s and store in s_new
s_new = np.diag(s)#change the shape of s as necessary 

# Because we are using 4 latent features and there are only 4 movies, 
# vt and vt_new are the same
vt_new = vt# change the shape of vt as necessary

In [16]:
# Check your matrices against the solution
assert u_new.shape == (20, 4), "Oops!  The shape of the u matrix doesn't look right. It should be 20 by 4."
assert s_new.shape == (4, 4), "Oops!  The shape of the sigma matrix doesn't look right.  It should be 4 x 4."
assert vt_new.shape == (4, 4), "Oops! The shape of the v transpose matrix doesn't look right.  It should be 4 x 4."
assert np.allclose(np.dot(np.dot(u_new, s_new), vt_new), user_movie_subset), "Oops!  Something went wrong with the dot product.  Your result didn't reproduce the original movie_user matrix."
print("That's right! The dimensions of u should be 20 x 4, and both v transpose and sigma should be 4 x 4.  The dot product of the three matrices how equals the original user-movie matrix!")

That's right! The dimensions of u should be 20 x 4, and both v transpose and sigma should be 4 x 4.  The dot product of the three matrices how equals the original user-movie matrix!


It turns out that the sigma matrix can actually tell us how much of the original variability in the user-movie matrix is captured by each latent feature.  The total amount of variability to be explained is the sum of the squared diagonal elements.  The amount of variability explained by the first component is the square of the first value in the diagonal.  The amount of variability explained by the second component is the square of the second value in the diagonal.   

`6.` Using the above information, can you determine the amount of variability in the original user-movie matrix that can be explained by only using the first two components? Use the cell below for your work, and then test your answer against the solution with the following cell.

In [30]:
total_var = np.sum(s**2) # Total variability here
var_exp_comp1_and_comp2 = s[0]**2 + s[1]**2 # Variability Explained by the first two components here
perc_exp = round(var_exp_comp1_and_comp2/total_var*100, 2) # Percent of variability explained by the first two components here

# Run the below to print your results
print("The total variance in the original matrix is {}.".format(total_var))
print("Ther percentage of variability captured by the first two components is {}%.".format(perc_exp))

The total variance in the original matrix is 5877.0.
Ther percentage of variability captured by the first two components is 98.55%.


In [31]:
assert np.round(perc_exp, 2) == 98.55, "Oops!  That doesn't look quite right.  You should have total variability as the sum of all the squared elements in the sigma matrix.  Then just the sum of the squared first two elements is the amount explained by the first two latent features.  Try again."
print("Yup!  That all looks good!")

Yup!  That all looks good!


`7.` Similar to the previous question, change the shapes of your u, sigma, and v transpose matrices.  However, this time consider only using the first 2 components to reproduce the user-movie matrix instead of all 4. After you have your matrices set up, check your matrices against the solution by running the tests.  The matrices should have the following dimensions:

$$ U_{n x k} $$

$$\Sigma_{k x k} $$

$$V^T_{k x m} $$

where:

1. n is the number of users
2. k is the number of latent features to keep (2 for this case)
3. m is the number of movies

In [33]:
s

array([ 75.48556738,   9.67425201,   7.42235236,   5.49969554])

In [32]:
k=3
np.diag(s[:k]) #this will pull out only the specified size

array([[ 75.48556738,   0.        ,   0.        ],
       [  0.        ,   9.67425201,   0.        ],
       [  0.        ,   0.        ,   7.42235236]])

In [34]:
# Change the dimensions of u, s, and vt as necessary to use four latent features
# update the shape of u and store in u_new
k = 2
u_2 = u[:,:k]# change the shape of u here

# update the shape of s and store in s_new
s_2 = np.diag(s[:k])#change the shape of s as necessary 

# Because we are using 2 latent features and there are only 4 movies, 
# vt and vt_new are the same
vt_2 = vt[:k,:]# change the shape of vt as necessary

In [35]:
# Check that your matrices are the correct shapes
assert u_2.shape == (20, 2), "Oops!  The shape of the u matrix doesn't look right. It should be 20 by 2."
assert s_2.shape == (2, 2), "Oops!  The shape of the sigma matrix doesn't look right.  It should be 2 x 2."
assert vt_2.shape == (2, 4), "Oops! The shape of the v transpose matrix doesn't look right.  It should be 2 x 4."
print("That's right! The dimensions of u should be 20 x 2, sigma should be 2 x 2, and v transpose should be 2 x 4. \n\n The question is now that we don't have all of the latent features, how well can we really re-create the original user-movie matrix?")

That's right! The dimensions of u should be 20 x 2, sigma should be 2 x 2, and v transpose should be 2 x 4. 

 The question is now that we don't have all of the latent features, how well can we really re-create the original user-movie matrix?


`8.` When using all 4 latent features, we saw that we could exactly reproduce the user-movie matrix.  Now that we only have 2 latent features, we might measure how well we are able to reproduce the original matrix by looking at the sum of squared errors from each rating produced by taking the dot product as compared to the actual rating.  Find the sum of squared error based on only the two latent features, and use the following cell to test against the solution. 

In [39]:
pred_ratings = np.dot(np.dot(u_2,s_2), vt_2)
pred_ratings, user_movie_subset

(array([[ 10.46939358,   9.03881747,   9.96601728,  10.39558706],
        [  8.57394283,   7.33732799,   9.10237708,   8.43627528],
        [  7.72063939,   6.48055583,  10.02663724,   7.44642893],
        [  9.51965759,   8.1567744 ,   9.95984596,   9.37883457],
        [  8.52709287,   7.29339826,   9.10813317,   8.38562191],
        [  5.86801737,   4.76039943,  10.00853852,   5.46357733],
        [  9.16529142,   7.82558125,   9.98769311,   8.99698805],
        [  8.1727267 ,   6.96220511,   9.13598032,   8.00377539],
        [  8.14151816,   6.87935565,   9.91483343,   7.90640917],
        [  7.48365698,   6.32349573,   9.11347406,   7.26756269],
        [  8.81092524,   7.49438809,  10.01554026,   8.61514153],
        [  9.78032386,   8.40010809,   9.94351101,   9.65937436],
        [  9.45314501,   8.08916774,  10.04380263,   9.30070086],
        [  7.22034399,   6.37477846,   4.83319403,   7.33691257],
        [  9.85436114,   8.46429062,  10.01019938,   9.73320075],
        [ 

In [38]:
user_movie_subset - pred_ratings

/opt/conda/lib/python3.6/site-packages/numexpr/cpuinfo.py:42: UserWarning: [Errno 12] Cannot allocate memory
  warnings.warn(str(e), UserWarning, stacklevel=stacklevel)


movie_id,73486,75314,68646,99685
user_id,,,,
265,-0.469394,0.961183,0.033983,-0.395587
1023,1.426057,-3.337328,-0.102377,1.563725
1683,0.279361,2.519444,-0.026637,-2.446429
6571,-0.519658,-0.156774,0.040154,0.621165
11639,1.472907,-2.293398,-0.108133,0.614378
13006,0.131983,-0.760399,-0.008539,0.536423
14076,-0.165291,0.174419,0.012307,0.003012
14725,1.827273,-1.962205,-0.135980,-0.003775
23548,-1.141518,1.120644,0.085167,0.093591


In [47]:
# Compute the dot product
pred_ratings = np.dot(np.dot(u_2,s_2), vt_2)# store the result of the dot product here

# Compute the squared error for each predicted vs. actual rating
sum_square_errs = round(np.sum(np.sum((user_movie_subset - pred_ratings)**2)),2)# compute the sum of squared differences from each prediction to each actual value here

In [48]:
sum_square_errs

85.340000000000003

In [49]:
# Check against the solution
assert np.round(sum_square_errs, 2) == 85.34, "Oops!  That doesn't look quite right.  You should return a single number for the whole matrix."
print("That looks right!  Nice job!")

That looks right!  Nice job!


At this point, you may be thinking . . . why would we want to choose a k that doesn't just give us back the full user-movie matrix with all the original ratings.  This is a good question.  One reason might be for computational reasons - sure, you may want to reduce the dimensionality of the data you are keeping, but really this isn't the main reason we would want to perform reduce k to lesser than the minimum of the number of movies or users.

Let's take a step back for a second.  In this example we just went through, your matrix was very clean.  That is, for every user-movie combination, we had a rating.  **There were no missing values.** But what we know from the previous lesson is that the user-movie matrix is full of missing values.  

A matrix similar to the one we just performed SVD on:

<img src="imgs/nice_ex.png" width="400" height="400">

The real world:

<img src="imgs/real_ex.png" width="400" height="400">


Therefore, if we keep all k latent features it is likely that latent features with smaller values in the sigma matrix will explain variability that is probably due to noise and not signal. Furthermore, if we use these "noisey" latent features to assist in re-constructing the original user-movie matrix it will potentially (and likely) lead to worse ratings than if we only have latent features associated with signal.   

`9.` Let's try introducing just a little of the real world into this example by performing SVD on a matrix with missing values.  Below I have added a new user to our matrix who hasn't rated all four of our movies.  Try performing SVD on the new matrix.  What happens?

In [51]:
# This line adds one nan value as the very first entry in our matrix
user_movie_subset.iloc[0, 0] = np.nan # no changes to this line

# Try svd with this new matrix
u, s, vt = np.linalg.svd(user_movie_subset, full_matrices=True)# Compute SVD on the new matrix with the single nan value

LinAlgError: SVD did not converge


**Write your response here.** 